<a href="https://colab.research.google.com/github/Matonice/30-Days-of-Transformer/blob/main/Fine_tuning_a_transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Building a paraphrase dectector** 

In [ ]:
%%capture
!pip install transformers
!pip install datasets
!pip install sentencepiece
!pip install evaluate

In [ ]:
#wrapping the output in collab cell
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
      white-space: pre-wrap;
    }

  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

**Loading in the dataset**

In [ ]:
from datasets import load_dataset
import evaluate
import numpy as np

raw_datasets = load_dataset("glue", "mrpc")
raw_datasets

Generating train split:   0%|          | 0/3668 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/408 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1725 [00:00<?, ? examples/s]

Dataset glue downloaded and prepared to /root/.cache/huggingface/datasets/glue/mrpc/1.0.0/dacbe3125aa31d7f70367a07a8a9e72a5a0bfeb5fc42e75c9db75b96da6053ad. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx'],
        num_rows: 1725
    })
})

In [ ]:
#checking the dataset
raw_dataset_train = raw_datasets['train']
raw_dataset_train[0]

{'sentence1': 'Amrozi accused his brother , whom he called " the witness " , of deliberately distorting his evidence .',
 'sentence2': 'Referring to him as only " the witness " , Amrozi accused his brother of deliberately distorting his evidence .',
 'label': 1,
 'idx': 0}

In [ ]:
#checking the featueres of the dataset
raw_dataset_train.features

{'sentence1': Value(dtype='string', id=None),
 'sentence2': Value(dtype='string', id=None),
 'label': ClassLabel(num_classes=2, names=['not_equivalent', 'equivalent'], id=None),
 'idx': Value(dtype='int32', id=None)}

**Preprocessing the dataset**

In [ ]:
from transformers import AutoTokenizer

checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

In [ ]:
#tokenizing a pair of sentence
inputs = tokenizer(["This is my first sentence", "This is my second sentence"])
print(inputs)

{'input_ids': [[101, 2023, 2003, 2026, 2034, 6251, 102], [101, 2023, 2003, 2026, 2117, 6251, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1]]}


In [ ]:
#decoding the inputs to see the format bert models tokenize a pair of sentence
tokenizer.convert_ids_to_tokens(inputs["input_ids"])

['[CLS]',
 'this',
 'is',
 'my',
 'first',
 'sentence',
 '[SEP]',
 'this',
 'is',
 'my',
 'second',
 'sentence',
 '[SEP]']

In [ ]:
#tokenizing the whole dataset
tokenized_datasets = tokenizer(
    raw_datasets["train"]["sentence1"],
    raw_datasets["train"]["sentence2"],
    padding=True,
    truncation=True
)

#the issue with this is that it returns a list of list as the values of the keys(input_ids, attention_mask, token_ids)
#so we are going to make use of another method for tokenization

In [ ]:
#defining a function to tokenize the dataset
def tokenize_function(example):
  return tokenizer(example["sentence1"], example["sentence2"], truncation=True)

In [ ]:
#applying the function to the entire dataset
tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
tokenized_datasets

  0%|          | 0/4 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3668
    })
    validation: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 408
    })
    test: Dataset({
        features: ['sentence1', 'sentence2', 'label', 'idx', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 1725
    })
})

In [ ]:
#applying  padding to each batch
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [ ]:
#connecting to my hugging face account
from huggingface_hub import notebook_login

notebook_login()

**Fine-tunning a model with the trainer api**

In [ ]:
from google.colab import output
output.enable_custom_widget_manager()

In [ ]:
#Defining our labels
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments("paraphrase_detector", evaluation_strategy="epoch", push_to_hub=True)


In [ ]:
#Defining a function to compute the evaluation metrics
def compute_metrics(eval_preds):
    metric = evaluate.load("glue", "mrpc")
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
!huggingface-cli login


        _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
        _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
        _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
        _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
        _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

        To login, `huggingface_hub` now requires a token generated from https://huggingface.co/settings/tokens .
        
Token: 
Login successful
Your token has been saved to /root/.huggingface/token
Authenticated through git-credential store but this isn't the helper defined on your machine.
You might have to re-authenticate when pushing to the Hugging Face Hub. Run the following command in your termin

In [ ]:
#Defining a trainer
from transformers import Trainer

trainer = Trainer(
    model,
    training_args,
    train_dataset = tokenized_datasets["train"],
    eval_dataset = tokenized_datasets["validation"],
    data_collator = data_collator,
    tokenizer = tokenizer,
    compute_metrics = compute_metrics
)

Cloning https://huggingface.co/abdulmatinomotoso/paraphrase_detector into local empty directory.


In [ ]:
trainer.train()

The following columns in the training set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running training *****
  Num examples = 3668
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1377


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.496818,0.848039,0.890071
2,0.329700,0.659903,0.855392,0.898451
3,0.138200,0.659903,0.855392,0.898451


The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8


Epoch,Training Loss,Validation Loss


Saving model checkpoint to paraphrase_detector/checkpoint-500
Configuration saved in paraphrase_detector/checkpoint-500/config.json
Model weights saved in paraphrase_detector/checkpoint-500/pytorch_model.bin
tokenizer config file saved in paraphrase_detector/checkpoint-500/tokenizer_config.json
Special tokens file saved in paraphrase_detector/checkpoint-500/special_tokens_map.json
tokenizer config file saved in paraphrase_detector/tokenizer_config.json
Special tokens file saved in paraphrase_detector/special_tokens_map.json
The following columns in the evaluation set don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: sentence2, sentence1, idx. If sentence2, sentence1, idx are not expected by `BertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 408
  Batch size = 8
Saving model checkpoint to paraphrase_detector/checkpoint-1000
Configuration saved in paraphrase_dete

TrainOutput(global_step=1377, training_loss=0.19204533507015364, metrics={'train_runtime': 220.9794, 'train_samples_per_second': 49.797, 'train_steps_per_second': 6.231, 'total_flos': 540800107631040.0, 'train_loss': 0.19204533507015364, 'epoch': 3.0})

In [ ]:
trainer.push_to_hub()

Saving model checkpoint to paraphrase_detector
Configuration saved in paraphrase_detector/config.json
Model weights saved in paraphrase_detector/pytorch_model.bin
tokenizer config file saved in paraphrase_detector/tokenizer_config.json
Special tokens file saved in paraphrase_detector/special_tokens_map.json
Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file pytorch_model.bin:   0%|          | 3.34k/418M [00:00<?, ?B/s]

Upload file runs/Aug21_21-42-37_7a5dcc910e58/events.out.tfevents.1661118383.7a5dcc910e58.71.0:  38%|###7      …

remote: Scanning LFS files for validity, may be slow...        
remote: LFS file scan complete.        
To https://huggingface.co/abdulmatinomotoso/paraphrase_detector
   2bf4c78..49072c5  main -> main

remote: LFS file scan complete.        
To https://huggingface.co/abdulmatinomotoso/paraphrase_detector
   2bf4c78..49072c5  main -> main

To https://huggingface.co/abdulmatinomotoso/paraphrase_detector
   49072c5..3d99dd2  main -> main

   49072c5..3d99dd2  main -> main



'https://huggingface.co/abdulmatinomotoso/paraphrase_detector/commit/49072c5c9d9d2463db5950726b1ea98559c6df30'